# --- Day 8: Playground ---

Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad.

You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground!

Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes.

Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes.

The Elves are trying to figure out **which junction boxes to connect** so that electricity can reach **every** junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input).

For example:

```
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
```

This list describes the position of 20 junction boxes, one per line. Each position is given as `X,Y,Z` coordinates. So, the first junction box in the list is at `X=162, Y=817, Z=812`.

To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are **as close together as possible** according to straight-line distance. In this example, the two junction boxes which are closest together are `162,817,812` and `425,690,689`.

By connecting these two junction boxes together, because electricity can flow between them, they become part of the same **circuit**. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.

Now, the two junction boxes which are closest together but aren't already directly connected are `162,817,812` and `431,825,988`. After connecting them, since `162,817,812` is already connected to another junction box, there is now a single circuit which contains **three** junction boxes and an additional 17 circuits which contain one junction box each.

The next two junction boxes to connect are `906,360,560` and `805,96,715`. After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.

The next two junction boxes are `431,825,988` and `425,690,689`. Because these two junction boxes were **already in the same circuit**, nothing happens!

This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.

After making the ten shortest connections, there are 11 circuits: one circuit which contains `5` junction boxes, one circuit which contains `4` junction boxes, two circuits which contain `2` junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces `40`.

Your list contains many junction boxes; connect together the `1000` pairs of junction boxes which are closest together. Afterward, **what do you get if you multiply together the sizes of the three largest circuits**?

In [2]:
# get inputs

with open("inputs/day_08.txt") as file:
    my_inputs = file.read().splitlines()
    
with open("inputs/day_08_test.txt") as file:
    test_inputs = file.read().splitlines()

In [3]:
test_inputs

['162,817,812',
 '57,618,57',
 '906,360,560',
 '592,479,940',
 '352,342,300',
 '466,668,158',
 '542,29,236',
 '431,825,988',
 '739,650,466',
 '52,470,668',
 '216,146,977',
 '819,987,18',
 '117,168,530',
 '805,96,715',
 '346,949,466',
 '970,615,88',
 '941,993,340',
 '862,61,35',
 '984,92,344',
 '425,690,689']

In [4]:
def get_junction_boxes(raw_input):
    junction_boxes = []
    for i in raw_input:
        coords = [int(n) for n in i.split(',')]
        box = {'id': i, 'position': tuple(coords)}
        junction_boxes.append(box)
    return junction_boxes

In [7]:
test_junction_boxes = get_junction_boxes(test_inputs)
test_junction_boxes[:5]

[{'id': '162,817,812', 'position': (162, 817, 812)},
 {'id': '57,618,57', 'position': (57, 618, 57)},
 {'id': '906,360,560', 'position': (906, 360, 560)},
 {'id': '592,479,940', 'position': (592, 479, 940)},
 {'id': '352,342,300', 'position': (352, 342, 300)}]

In [21]:
import math

def calculate_distances(junction_boxes):
    distances = set()
    for box in junction_boxes:
        for other_box in junction_boxes:
            if box['id'] != other_box['id']:
                distance = math.dist(box['position'], other_box['position'])
                distance_between = sorted([box['id'], other_box['id']])
                distance_record = (distance, tuple(distance_between))
                distances.add(distance_record)
    sorted_distances = sorted(list(distances))
    return sorted_distances

In [23]:
test_distances = calculate_distances(test_junction_boxes)
test_distances[:10]

[(316.90219311326956, ('162,817,812', '425,690,689')),
 (321.560258738545, ('162,817,812', '431,825,988')),
 (322.36935338211043, ('805,96,715', '906,360,560')),
 (328.11888089532425, ('425,690,689', '431,825,988')),
 (333.6555109690233, ('862,61,35', '984,92,344')),
 (338.33858780813046, ('117,168,530', '52,470,668')),
 (344.3893145845266, ('819,987,18', '941,993,340')),
 (347.59890678769403, ('739,650,466', '906,360,560')),
 (350.786259708102, ('346,949,466', '425,690,689')),
 (352.936254867646, ('906,360,560', '984,92,344'))]

In [55]:
def connect_boxes_into_circuits(distances, num_connections):
    circuits = []
    connection_count = 0
    while connection_count < num_connections:
        for record in distances:
            print(circuits)
            pair = record[1]
            if circuits:
                is_new = True
                for circuit in circuits:
                    if (pair[0] in circuit) and (pair[1] in circuit):
                        break
                    if (pair[0] in circuit) or (pair[1] in circuit):
                        circuit.add(pair[0])
                        circuit.add(pair[1])
                        connection_count += 1
                        is_new = False
                        break
                if is_new:
                    new_circuit = set(pair)
                    connection_count += 1
                    circuits.append(new_circuit)
            else:
                new_circuit = set(pair)
                connection_count += 1
                circuits.append(new_circuit)
            if connection_count == num_connections:
                break
    return sorted(circuits, key=len, reverse=True)

In [ ]:
test_circuits = connect_boxes_into_circuits(test_distances, 10)
test_circuits

[]
[{'162,817,812', '425,690,689'}]
[{'162,817,812', '425,690,689', '431,825,988'}]
[{'162,817,812', '425,690,689', '431,825,988'}, {'906,360,560', '805,96,715'}]
[{'162,817,812', '425,690,689', '431,825,988'}, {'906,360,560', '805,96,715'}, {'431,825,988', '425,690,689'}]
[{'162,817,812', '425,690,689', '431,825,988'}, {'906,360,560', '805,96,715'}, {'431,825,988', '425,690,689'}, {'862,61,35', '984,92,344'}]
[{'162,817,812', '425,690,689', '431,825,988'}, {'906,360,560', '805,96,715'}, {'431,825,988', '425,690,689'}, {'862,61,35', '984,92,344'}, {'52,470,668', '117,168,530'}]
[{'162,817,812', '425,690,689', '431,825,988'}, {'906,360,560', '805,96,715'}, {'431,825,988', '425,690,689'}, {'862,61,35', '984,92,344'}, {'52,470,668', '117,168,530'}, {'941,993,340', '819,987,18'}]
[{'162,817,812', '425,690,689', '431,825,988'}, {'739,650,466', '906,360,560', '805,96,715'}, {'431,825,988', '425,690,689'}, {'862,61,35', '984,92,344'}, {'52,470,668', '117,168,530'}, {'941,993,340', '819,987,18

[{'162,817,812', '346,949,466', '425,690,689', '431,825,988'},
 {'739,650,466', '805,96,715', '906,360,560', '984,92,344'},
 {'425,690,689', '431,825,988'},
 {'862,61,35', '984,92,344'},
 {'117,168,530', '52,470,668'},
 {'819,987,18', '941,993,340'}]

In [54]:
test_distances[:10]

[(316.90219311326956, ('162,817,812', '425,690,689')),
 (321.560258738545, ('162,817,812', '431,825,988')),
 (322.36935338211043, ('805,96,715', '906,360,560')),
 (328.11888089532425, ('425,690,689', '431,825,988')),
 (333.6555109690233, ('862,61,35', '984,92,344')),
 (338.33858780813046, ('117,168,530', '52,470,668')),
 (344.3893145845266, ('819,987,18', '941,993,340')),
 (347.59890678769403, ('739,650,466', '906,360,560')),
 (350.786259708102, ('346,949,466', '425,690,689')),
 (352.936254867646, ('906,360,560', '984,92,344'))]